<a href="https://colab.research.google.com/github/alexsabrasil/ALEXSA003_prompt_alert/blob/main/Treinamento_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Treinamento SQL em Python**


####**Pergunta: Como você faria para encontrar registros duplicados em uma tabela no banco de dados?**

**Respondendo:** Para encontrar registros duplicados, usaria a cláusula **GROUP BY** em conjunto com a função **COUNT()**. Por exemplo, se eu quiser encontrar duplicados em uma coluna como email, faria algo assim:

In [13]:
import sqlite3
import pandas as pd

# Criar um banco de dados em memória
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Criar e popular a tabela "tabela" para o primeiro exemplo
cursor.execute("""
CREATE TABLE tabela (
    email TEXT
);
""")

# Inserir dados fictícios
cursor.executemany("INSERT INTO tabela (email) VALUES (?)", [
    ("email1@example.com",),
    ("email2@example.com",),
    ("email1@example.com",),  # Duplicado
    ("email3@example.com",),
    ("email2@example.com",),  # Duplicado
])

# Query para encontrar duplicados
query1 = """
SELECT email, COUNT(*) AS quantidade
FROM tabela
GROUP BY email
HAVING COUNT(*) > 1;
"""
result1 = pd.read_sql_query(query1, conn)
print("Duplicados encontrados:")
print(result1)

Duplicados encontrados:
                email  quantidade
0  email1@example.com           2
1  email2@example.com           2


###Isso me mostraria os valores duplicados na coluna email e quantas vezes eles aparecem.

####Cenário: **Um gestor pede que você escreva uma consulta SQL para listar os nomes de clientes que fizeram mais de 5 compras em um mês específico.**

####**Explique enquanto escreve:**
Estou juntando as tabelas clientes e vendas pela *chave estrangeira* **id_cliente**. *Filtrei as vendas para o mês específico* usando a função **DATE_FORMAT**. Por fim, *agrupei os clientes* e usei **HAVING** para exibir apenas aqueles com mais de **5 compras**.

In [14]:
import sqlite3
import pandas as pd

# Criar as tabelas "clientes" e "vendas" para o segundo exemplo
cursor.execute("""
CREATE TABLE clientes (
    id_cliente INTEGER PRIMARY KEY,
    nome_cliente TEXT
);
""")
cursor.execute("""
CREATE TABLE vendas (
    id_venda INTEGER PRIMARY KEY,
    id_cliente INTEGER,
    data_venda TEXT,
    FOREIGN KEY (id_cliente) REFERENCES clientes (id_cliente)
);
""")

# Inserir dados fictícios
cursor.executemany("INSERT INTO clientes (id_cliente, nome_cliente) VALUES (?, ?)", [
    (1, "João"),
    (2, "Maria"),
    (3, "Ana"),
])
cursor.executemany("INSERT INTO vendas (id_venda, id_cliente, data_venda) VALUES (?, ?, ?)", [
    (1, 1, "2024-11-01"),
    (2, 1, "2024-11-02"),
    (3, 1, "2024-11-03"),
    (4, 2, "2024-11-01"),
    (5, 2, "2024-11-02"),
    (6, 2, "2024-11-03"),
    (7, 2, "2024-11-04"),
    (8, 2, "2024-11-05"),
    (9, 2, "2024-11-06"),
])

# Query para encontrar clientes com mais de 5 compras em novembro de 2024
query2 = """
SELECT c.nome_cliente, COUNT(v.id_venda) AS total_compras
FROM clientes c
JOIN vendas v ON c.id_cliente = v.id_cliente
WHERE strftime('%Y-%m', v.data_venda) = '2024-11'
GROUP BY c.nome_cliente
HAVING total_compras > 5;
"""
result2 = pd.read_sql_query(query2, conn)
print("\nClientes com mais de 5 compras:")
print(result2)

# Fechar a conexão
conn.close()


Clientes com mais de 5 compras:
  nome_cliente  total_compras
0        Maria              6
